# RAG  based on mistral 7b

## Loading the model

In [1]:
!pip install -q torch datasets
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                transformers==4.31.0 \
                trl==0.4.7

In [2]:
pip uninstall bitsandbytes

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------- 101.6/101.6 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.40.2
    Uninstalling bitsandbytes-0.40.2:
      Successfully uninstalled bitsandbytes-0.40.2
Note: you may need to restart the kernel to use updated packages.


In [1]:
import transformers
import huggingface_hub

In [2]:
import bitsandbytes


In [9]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 9.0/9.0 MB 19.2 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 23.5 MB/s eta 0:00:00
     ------------------------------------- 388.9/388.9 kB 23.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
#################################################################
# Tokenizer（pip install --user transformers==4.34.0）
#################################################################
model_name='mistralai/Mistral-7B-Instruct-v0.1'

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [4]:
#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [5]:
from transformers import BitsAndBytesConfig
import torch
#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [6]:
import torch

print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.1.0+cu121
Is CUDA enabled? True


In [7]:
from transformers import AutoModelForCausalLM
#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## RAG based on mistral

#### Create LLM Chain

In [8]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import LLMChain
import transformers
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    repetition_penalty=1.2,
    return_full_text=True,
    max_new_tokens=700,
)
prompt_template = """
### [INST] 
Instruction: 
Extract and answer from your own knowledge. Ensure the response is concise, without duplicates, focusing solely on crucial details. 
             
           

### Example
Question: What is the sun?
Answer: A star at the center of the solar system.

Question: Who was the first person to walk on the moon?
Answer: Neil Armstrong.





### QUESTION:
{question} 

[/INST]
 """

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [9]:
import pandas as pd

# Assume llm_chain is already defined

def process_dataframe(file_path, output_file_path):
    # Load data from Excel file
    df = pd.read_excel(file_path)

    # Iterate through each row of the DataFrame
    for index, row in df.iterrows():
        # Extract context and question from each row, ensuring they are of string type
    
        question = str(row['Competency Question'])

        # Call the llm_chain method to get the result
        result = llm_chain.invoke({ "question": question})

        # Save the result in the 'Predicted Answer' column
        df.at[index, 'Predicted Answer'] = result

        # Print the obtained answer
        print(f"Answer for row {index + 1}: {result}")

        # Print the current row being processed to provide progress feedback
        print(f"Processed {index + 1} / {len(df)} rows")

    # Write the modified DataFrame back to a new Excel file
    df.to_excel(output_file_path, index=False)



# Processing the file
process_dataframe('C:/Users/crown/Desktop/ISE/praktikum docu/train_8/train_8.xlsx', 
                  'C:/Users/crown/Desktop/ISE/praktikum docu/train_8/train_result_mistral_withoutkg.xlsx')   
### These are the input file and out put file




C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Answer for row 1: {'question': 'Who is working in the Computational Materials Science field?', 'text': '\nThere are many scientists and researchers who work in the computational materials science field. Some notable figures include:\n\n* John Goodenough - a professor emeritus at Massachusetts Institute of Technology (MIT) and inventor of the lithium-ion battery.\n* Mildred Dresselhaus - a professor of physics and chemistry at MIT and recipient of the Nobel Prize in Physics in 2019 for her work on carbon nanotubes.\n* Shuji Nakamura - a professor of biology at Kyoto University and winner of the Nobel Prize in Physiology or Medicine in 2016 for his discovery of the molecular mechanism that controls stem cell self-renewal.\n* Jianqing Zhang - a professor of materials science and engineering at Stanford University and pioneer in the development of high-temperature superconductors.'}
Processed 1 / 38 rows


C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Answer for row 2: {'question': 'What are the research projects associated to EMMO?', 'text': '\nEMMO stands for "ElectroMagnetic Manipulation of Organisms." It refers to a field of study that focuses on using electromagnetic fields to manipulate living organisms such as cells, tissues, organs, and even entire organisms like animals or plants. Some potential research projects in this area could include studying how electromagnetic fields affect cellular processes, investigating the use of electromagnetic therapy for medical treatments, exploring the effects of electromagnetic pollution on human health, and examining the potential applications of electromagnetism in agriculture or environmental conservation efforts.'}
Processed 2 / 38 rows


C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


...

Answer for row 36: {'question': 'Who is the creators of the "BAM reference data"?', 'text': ' The BAM (Binary Alignment Map) reference data is a standard format for storing genomic sequence alignments in a compressed binary format. It was developed by the Genome Reference Consortium (GRC), which is an international organization that coordinates efforts to improve the quality and accessibility of human genome assemblies and related resources.'}
Processed 36 / 38 rows


C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Answer for row 37: {'question': 'What are the datasets published by "BAM"?', 'text': '\n"BAM" stands for Berkeley Astronomy & Astrophysics Laboratory. They publish several datasets related to their research in astronomy and astrophysics. Some examples include:\n\n1. The Berkeley Open Sky Survey (BOSS): This dataset contains photometric measurements of over 850 million galaxies in the redshift range z = 0.2 to 3.4. It was used to map the distribution of matter in the universe and study galaxy evolution.\n2. The Berkeley Deep Sky Survey (BDSS): This survey covers a large portion of the visible sky and provides high-resolution images of stars, galaxies, and other celestial objects.\n3. The Berkeley Variable Star Database (BVSD): This database contains information about variable stars observed by amateur astronomers around the world. It includes data on light curves, spectra, and other properties of these stars.\n4. The Berkeley Cosmic Background Explorer (COBE) Data Archive: COBE was a sa

C:\Users\crown\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Answer for row 38: {'question': 'Average', 'text': '1. What is the average temperature in New York City during summer months?\n2. How many hours are there in a day?\n3. What is the capital city of France?\n4. What is the largest planet in our solar system?\n5. What is the smallest country in the world by land area?'}
Processed 38 / 38 rows


In [10]:
df=pd.read_excel('C:/Users/crown/Desktop/ISE/praktikum docu/train_8/train_result_mistral_withoutkg.xlsx')

In [11]:
df['Predicted_Answer'] = df['Predicted Answer'].astype(str)

In [12]:
import re

# Add a new column 'answer_text'
df['answer_text'] = ''

# Use regular expression to match the content after 'text':
text_pattern = re.compile(r"'text':(.+?)}", re.DOTALL)

# Iterate through DataFrame and extract content
for index, row in df.iterrows():
    predicted_answer = row['Predicted_Answer']
    match = text_pattern.search(predicted_answer)
    if match:
        # Extract matched content
        answer_text = match.group(1)
    else:
        answer_text = "no answer"  # Set default value if no match is found

    # Store the extracted content in the new column
    df.at[index, 'answer_text'] = answer_text


In [14]:

df['answer_text'] = df['answer_text'].str.replace('\n', ' ', regex=False)

df['answer_text'] = df['answer_text'].str.replace(r'\*\s*', ' ', regex=True)

df['answer_text'] = df['answer_text'].str.replace(r'\\n', ' ', regex=True)

df.to_excel('C:/Users/crown/Desktop/ISE/praktikum docu/train_8/train_result_mistral_withoutkg.xlsx', index=False)